In [ ]:
import pandas as pd
import pickle
import re

In [ ]:
def load_cache(cache_fname):
    with open(cache_fname, 'rb') as fp:
        cache = pickle.load(fp)
    return cache

def retrieve_from_cache(cache, question, answer, narrative):

    """
    Use question, answer and narrative to retrieve all associated values
    Return failure if key not found
    """

    key = (question.lower(), answer.lower(), narrative.lower())
    try:
        return cache[key]
    except:
        return {'message': 'Key not found'}

In [ ]:
cache = load_cache('../artifacts/human_eval_cache.pkl')

In [ ]:
def remove_punctuation(text):
    result = re.sub(r'[^\w\s]', '', text)
    return result

In [ ]:
def overall_avg_likert(df, cache):
    likerts = []
    for idx, row in df.iterrows():
        question = row['question']
        try:
            answer = remove_punctuation(row['predicted_answer'])
        except:
            answer = remove_punctuation(row['answer'])
        narrative = row['narrative']
        key = (question, answer, narrative)
        info = retrieve_from_cache(cache, key[0], key[1], key[2])
        likertscores = info['val_annotations']
        likert = sum(likertscores) / len(likertscores)
        likerts.append(likert)
    print(f'Overall avg Likert for all answers {round(sum(likerts) / len(likerts), 2)}')

In [ ]:
def overall_avg_binary_likert(df, cache):
    likerts = []
    for idx, row in df.iterrows():
        question = row['question']
        try:
            answer = remove_punctuation(row['predicted_answer'])
        except:
            answer = remove_punctuation(row['answer'])
        narrative = row['narrative']
        key = (question, answer, narrative)
        info = retrieve_from_cache(cache, key[0], key[1], key[2])
        likertscores = info['val_annotations']
        binary_likertscores = [0 if x < 1 else 1 for x in likertscores]
        likert = sum(binary_likertscores) / len(binary_likertscores)
        likerts.append(likert)
    print(f'Overall avg binary Likert for all answers: {round(sum(likerts) / len(likerts), 2)}')

In [ ]:
def get_all_numbers(df, cache):
    overall_avg_likert(df, cache)
    overall_avg_binary_likert(df, cache)

In [ ]:
def evaluate_df_by_onto(df, cache):
    
    conseq_df = df[df['onto'] == 'Consequence']
    print('Consequence')
    get_all_numbers(conseq_df, cache)
    
    goal_df = df[df['onto'] == 'Goal seeking']
    print('Goal seeking')
    get_all_numbers(goal_df, cache)
    
    reac_df = df[df['onto'] == 'Reactionary']
    print('Reactionary')
    get_all_numbers(reac_df, cache)
    
    desire_df = df[df['onto'] == 'Desire']
    print('Desire')
    get_all_numbers(desire_df, cache)
    
    other_df = df[df['onto'] == 'Other']
    print('Other')
    get_all_numbers(other_df, cache)

In [ ]:
ontology_df = pd.read_csv('../artifacts/hidden_test_set_ontology.csv')

In [ ]:
meta_to_ontology_dict = {}
for idx, row in ontology_df.iterrows():
    meta_to_ontology_dict[row['question_meta']] = row['Ontology']

In [ ]:
def add_onto_to_df(df, meta_to_ontology_dict):
    ontos = []
    for idx, row in df.iterrows():
        try:
            ontos.append(meta_to_ontology_dict[row['question_meta']])
        except:
            ontos.append(meta_to_ontology_dict[row['meta']])
    df['onto'] = ontos
    return df

In [ ]:
onto_count_dict = ontology_df['Ontology'].value_counts().to_dict()

In [ ]:
onto_count_dict

In [ ]:
t5_df = pd.read_csv('../artifacts/model_predictions/t5base_w_n_separator.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
evaluate_df_by_onto(impl_t5_df, cache)

In [ ]:
t5_knowl_df = pd.read_csv('../artifacts/model_predictions/t5base_w_n_separator_w_knowl.csv')
t5_knowl_df = add_onto_to_df(t5_knowl_df, meta_to_ontology_dict)
get_all_numbers(t5_knowl_df, cache)

In [ ]:
impl_t5_knowl_df = t5_knowl_df[t5_knowl_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_knowl_df, cache)

In [ ]:
evaluate_df_by_onto(impl_t5_knowl_df, cache)

In [ ]:
t511b_df = pd.read_csv('../artifacts/model_predictions/t511b_w_n_separator.csv')
t511b_df = add_onto_to_df(t511b_df, meta_to_ontology_dict)
get_all_numbers(t511b_df, cache)

In [ ]:
impl_t511b_df = t511b_df[t511b_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t511b_df, cache)

In [ ]:
evaluate_df_by_onto(impl_t511b_df, cache)

In [ ]:
# this file is the t511b with top 3 diverse comet verbalized
t511b_knowl_df = pd.read_csv('../artifacts/model_predictions/t511b_w_n_separator_w_knowl.csv')
t511b_knowl_df = add_onto_to_df(t511b_knowl_df, meta_to_ontology_dict)
get_all_numbers(t511b_knowl_df, cache)

In [ ]:
impl_t511b_knowl_df = t511b_knowl_df[t511b_knowl_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t511b_knowl_df, cache)

In [ ]:
evaluate_df_by_onto(impl_t511b_knowl_df, cache)

In [ ]:
gpt3_df = pd.read_csv('../artifacts/model_predictions/gpt3.csv')
gpt3_df = add_onto_to_df(gpt3_df, meta_to_ontology_dict)
get_all_numbers(gpt3_df, cache)

In [ ]:
impl_gpt3_df = gpt3_df[gpt3_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_gpt3_df, cache)

In [ ]:
evaluate_df_by_onto(impl_gpt3_df, cache)

In [ ]:
gpt3_knowl_df = pd.read_csv('../artifacts/model_predictions/gpt3_w_knowl.csv')
gpt3_knowl_df = add_onto_to_df(gpt3_knowl_df, meta_to_ontology_dict)
get_all_numbers(gpt3_knowl_df, cache)

In [ ]:
impl_gpt3_knowl_df = gpt3_knowl_df[gpt3_knowl_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_gpt3_knowl_df, cache)

In [ ]:
evaluate_df_by_onto(impl_gpt3_knowl_df, cache)

# Model Setup

## Base

In [ ]:
print('Gtup top3')
t5_df = pd.read_csv('../artifacts/model_predictions/t5base_tup_top3_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gtupsep top3')
t5_df = pd.read_csv('../artifacts/model_predictions/t5base_tupsep_top3_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top1')
t5_df = pd.read_csv('../artifacts/model_predictions/t5base_verb_top1_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top5 diverse')
t5_df = pd.read_csv('../artifacts/model_predictions/t5base_verb_top5_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top3 original')
t5_df = pd.read_csv('../artifacts/model_predictions/t5base_verb_top3_original.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top3 diverse')
t5_df = pd.read_csv('../artifacts/model_predictions/t5base_verb_top3_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top3 reranked')
t5_df = pd.read_csv('../artifacts/model_predictions/t5base_verb_top3_reranked.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('T5 Appendix D.3 format - no separator')
t5_df = pd.read_csv('../artifacts/model_predictions/t5base.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

## 11B

In [ ]:
print('Gtup top3')
t5_df = pd.read_csv('../artifacts/model_predictions/t511b_tup_top3_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gtupsep top3')
t5_df = pd.read_csv('../artifacts/model_predictions/t511b_tupsep_top3_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top1')
t5_df = pd.read_csv('../artifacts/model_predictions/t511b_verb_top1_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top5 diverse')
t5_df = pd.read_csv('../artifacts/model_predictions/t511b_verb_top5_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top3 original')
t5_df = pd.read_csv('../artifacts/model_predictions/t511b_verb_top3_original.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top3 diverse')
t5_df = pd.read_csv('../artifacts/model_predictions/t511b_verb_top3_diverse.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('Gverb. top3 reranked')
t5_df = pd.read_csv('../artifacts/model_predictions/t511b_verb_top3_reranked.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)

In [ ]:
print('T5 Appendix D.3 format - no separator')
t5_df = pd.read_csv('../artifacts/model_predictions/t511b.csv')
t5_df = add_onto_to_df(t5_df, meta_to_ontology_dict)
get_all_numbers(t5_df, cache)

In [ ]:
impl_t5_df = t5_df[t5_df['is_ques_answerable'] == 'Not Answerable']
get_all_numbers(impl_t5_df, cache)